# NORN initial analysis

Here we look at ways of analysing the NORN corpus

In [146]:
import dhlab as dh

from collections import Counter
import pandas as pd

import dhlab.api.dhlab_api as api
dh.css()

In [166]:
import requests
def find_urns(
    dhlabids= None,
    mode='json'): 
   
    params = locals()
    r = requests.post(api.BASE_URL + "/find_urn", json=params)
    if r.status_code == 200:
        res = r.json()
    else:
        res = {}
    return res

Get the corpus

In [16]:
norn = pd.read_excel('konsensuskorpus_master.xlsx').fillna('')
norn

,Verk,Forfatter,Utgivelsesår,Jæger 1896,Bing 1904,Hambro 1912,Paasche 1932,Beyer 1933,Elster 1935,Liestøl & Stang 1938,Dahl 1981,Beyer & Beyer 1996,Fidjestøl 1998,Andersen 2012,oaiid/URN,sidetall,flere utgivelser,kommentar
0,[Hvorfor har Norge ikke sit eget National-Sprog?],"Lundh, G. F.",1807,,,,,,,,,"x, 154",,,http://urn.nb.no/URN:NBN:no-nb_digibok_2011111...,21-49,,Kompendium. Starten på latin. Kan diskutere hv...
1,Luren,"Hansen, M.",1819,,"z, 75 sentimental-romantisk",,,,,,,,"x, 219",,http://urn.nb.no/URN:NBN:no-nb_digibok_2007082...,89-98,,Overlapp siste side. Opprinnelig i Morgenbladet
2,Anmeldelse av A. Faye: Norges Historie …,"Munch, P. A.",1832,,,,,,,,,"x, 155",,,http://urn.nb.no/URN:NBN:no-nb_digibok_2009100...,11-26,,"Samlede Afhandlinger, 1873. Opprinnelig i Maan..."
3,Henrik Wergelands Digtekunst og Polemik,"Welhaven, J. S.",[1832],,,,,,,,,"x, 158",,,http://urn.nb.no/URN:NBN:no-nb_digibok_2013110...,,,
4,Samlinger til det norske Folks Sprog og Histor...,"Berg, J. K.",[1833–],,,,,,"x, 182",,,,,,Tidsskrift,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,Familien Sterk,"Qvigstad, J. og Sandberg, G.",[1887],,,,,,"x, 192",,,,,,http://urn.nb.no/URN:NBN:no-nb_digibok_2021082...,124-128,,Overlapp
313,Lappiske eventyr og folkesagn,"Qvigstad, J. og Sandberg, G.",1887,,,,,,"x, 199",,,,,,http://urn.nb.no/URN:NBN:no-nb_digibok_2021082...,,,
314,Tvivl og Haab,"Ibsen, H.",[1892],,,,,,,,"x, 135",,,,http://urn.nb.no/URN:NBN:no-nb_digibok_2011062...,26-28,,Overlapp
315,[Segner fraa Vaagaa],"Kleiven, I.",[1894],,,"x, 61 (120)",,,,,,,,,http://urn.nb.no/URN:NBN:no-nb_digibok_2006112...,,,


In [17]:
#norn.fillna('') #.groupby(norn.columns[3:14]).agg(' '.join)

In [20]:
def convert(c):
    try:
        parts = c.split(',')
    except:
        parts = ['u']
    return parts[0]

In [24]:
classes = norn[norn.columns[3:14]].applymap(convert)

In [58]:
norn.columns[3:14]

Index(['Jæger 1896', 'Bing 1904', 'Hambro 1912', 'Paasche 1932', 'Beyer 1933',
       'Elster 1935', 'Liestøl & Stang 1938', 'Dahl 1981',
       'Beyer & Beyer 1996', 'Fidjestøl 1998', 'Andersen 2012'],
      dtype='object')

Pick out a subframe of the classifications

In [80]:
nornsubframe = norn[list(norn.columns[3:14])].applymap(convert)

In [88]:
urnframe = nornsubframe.applymap(convert).merge(norn[['oaiid/URN']], left_index=True, right_index=True)
urnframe.sample(5)

,Jæger 1896,Bing 1904,Hambro 1912,Paasche 1932,Beyer 1933,Elster 1935,Liestøl & Stang 1938,Dahl 1981,Beyer & Beyer 1996,Fidjestøl 1998,Andersen 2012,oaiid/URN
78,,,,,x,,,,,,,http://urn.nb.no/URN:NBN:no-nb_digibok_2010111...
159,,,x,,,x/y,y,y,,,,http://urn.nb.no/URN:NBN:no-nb_digibok_2014061...
266,,,,,,x,,,,,,http://urn.nb.no/URN:NBN:no-nb_digibok_2016110...
130,w,,x,,,,,,,,,http://urn.nb.no/URN:NBN:no-nb_digibok_2019110...
132,,,x,,w,x,x,,w,z,x,http://urn.nb.no/URN:NBN:no-nb_digibok_2008111...


Add class counts, how often a work is classified

In [86]:
class_counts = pd.DataFrame([(i,Counter([v for v in r if v != ''])) for i, r in nornsubframe.iterrows()],columns = [0,'counts']).set_index(0)

In [93]:
class_counts

,counts
0,
0,{'x': 1}
1,"{'z': 1, 'x': 1}"
2,{'x': 1}
3,{'x': 1}
4,{'x': 1}
...,...
312,{'x': 1}
313,{'x': 1}
314,{'x': 1}


In [94]:
c = class_counts.copy()

In [99]:
c.counts.values

array([Counter({'x': 1}), Counter({'z': 1, 'x': 1}), Counter({'x': 1}),
       Counter({'x': 1}), Counter({'x': 1}), Counter({'x': 7}),
       Counter({'x': 1}), Counter({'z': 1, 'x': 1}), Counter({'w': 2}),
       Counter({'z': 2, 'x': 2}), Counter({'x': 1}), Counter({'x': 2}),
       Counter({'x': 1}), Counter({'x': 2}), Counter({'x': 1}),
       Counter({'x': 2, 'w': 1, 'x/y': 1}), Counter({'x': 3}),
       Counter({'x': 2}), Counter({'x/y': 1, 'x': 1}), Counter({'x': 1}),
       Counter({'x': 2}), Counter({'x': 1, 'w': 1}), Counter({'x': 1}),
       Counter({'x': 1}), Counter({'x': 1}), Counter({'x': 1}),
       Counter({'w': 1, 'x': 1}), Counter({'x': 1}), Counter({'x': 1}),
       Counter({'w': 1}), Counter({'x': 2}), Counter({'x': 1}),
       Counter({'x': 2, 'x/y': 1}),
       Counter({'y': 2, 'w': 1, 'x': 1, 'x/y': 1}), Counter({'w': 1}),
       Counter({'x': 5, 'y': 1, 'x/y': 1}), Counter({'z': 1, 'x': 1}),
       Counter({'x': 1}), Counter({'x': 1}), Counter({'x': 5, 'y': 1}

Add counts as columns

In [103]:
for v in ['x','y','z', 'w', 'x/y', 'w/y']:
    c[v] = c.counts.apply(lambda f: f[v])


In [104]:
c.sort_values(by='x/y')

,counts,x,y,w,x/y,w/y,z
0,,,,,,,
0,{'x': 1},1,0,0,0,0,0
201,{'x': 3},3,0,0,0,0,0
200,{'x': 1},1,0,0,0,0,0
199,{'x': 2},2,0,0,0,0,0
198,{'x': 1},1,0,0,0,0,0
...,...,...,...,...,...,...,...
32,"{'x': 2, 'x/y': 1}",2,0,0,1,0,0
117,"{'x': 1, 'x/y': 1}",1,0,0,1,0,0
254,{'x/y': 1},0,0,0,1,0,0


Merge data with the books

In [91]:
class_counts.merge(urnframe, left_index =True, right_index = True)[['counts','oaiid/URN']]

,counts,oaiid/URN
0,,
0,{'x': 1},http://urn.nb.no/URN:NBN:no-nb_digibok_2011111...
1,"{'z': 1, 'x': 1}",http://urn.nb.no/URN:NBN:no-nb_digibok_2007082...
2,{'x': 1},http://urn.nb.no/URN:NBN:no-nb_digibok_2009100...
3,{'x': 1},http://urn.nb.no/URN:NBN:no-nb_digibok_2013110...
4,{'x': 1},Tidsskrift
...,...,...
312,{'x': 1},http://urn.nb.no/URN:NBN:no-nb_digibok_2021082...
313,{'x': 1},http://urn.nb.no/URN:NBN:no-nb_digibok_2021082...
314,{'x': 1},http://urn.nb.no/URN:NBN:no-nb_digibok_2011062...


In [108]:
consensus = c.merge(urnframe, left_index=True, right_index=True)[['x','y','z', 'w', 'x/y', 'w/y', 'oaiid/URN']]

In [112]:
consensus["oaiid/URN"] = consensus["oaiid/URN"].apply(lambda x: [y.strip() for y in x.split('+')])

In [120]:
consensus.sort_values(by="x", ascending = False).head(5).style

,x,y,z,w,x/y,w/y,oaiid/URN
0,,,,,,,
5,7,0,0,0,0,0,['http://urn.nb.no/URN:NBN:no-nb_digibok_2009061912006']
35,5,1,0,0,1,0,['http://urn.nb.no/URN:NBN:no-nb_digibok_2019121148002']
57,5,1,0,0,0,1,"['http://urn.nb.no/URN:NBN:no-nb_digibok_2012062813003', 'http://urn.nb.no/URN:NBN:no-nb_digibok_2012061313001']"
39,5,1,0,0,0,0,"['http://urn.nb.no/URN:NBN:no-nb_digibok_2008103012001', 'http://urn.nb.no/URN:NBN:no-nb_digibok_2015061929001']"
164,5,0,0,2,0,0,['http://urn.nb.no/URN:NBN:no-nb_digibok_2014072108075']


In [122]:
norn_consensus = consensus.merge(norn[["Verk","Forfatter"]], left_index=True, right_index=True)

In [125]:
norn_consensus.to_excel('norn_consensus.xlsx')

In [126]:
norn_consensus[norn_consensus.x > 4]

,x,y,z,w,x/y,w/y,oaiid/URN,Verk,Forfatter
0,,,,,,,,,
5,7,0,0,0,0,0,[http://urn.nb.no/URN:NBN:no-nb_digibok_200906...,Norske Sagn,"Faye, A."
35,5,1,0,0,1,0,[http://urn.nb.no/URN:NBN:no-nb_digibok_201912...,"Samling av Sange, Folkeviser og Stev i norske ...","Moe, J."
39,5,1,0,0,0,0,[http://urn.nb.no/URN:NBN:no-nb_digibok_200810...,Norske Folkeeventyr*,"Asbjørnsen, P. C. og Moe, J."
57,5,1,0,0,0,1,[http://urn.nb.no/URN:NBN:no-nb_digibok_201206...,Norske Huldre-Eventyr og Folkesagn*,"Asbjørnsen, P. C."
164,5,0,0,2,0,0,[http://urn.nb.no/URN:NBN:no-nb_digibok_201407...,Norske folkeviser*,"Landstad, M. B. [og Crøger, O.]"
202,5,0,1,0,0,0,[http://urn.nb.no/URN:NBN:no-nb_digibok_201106...,Hærmendene på Helgeland,"Ibsen, H."


In [131]:
urns = [ur.split('/')[-1] for u in norn_consensus['oaiid/URN'].values for ur in u]

In [139]:
urns = [u for u in urns if u.startswith('URN')]
urns[:5]

['URN:NBN:no-nb_digibok_2011111605037',
 'URN:NBN:no-nb_digibok_2007082703001',
 'URN:NBN:no-nb_digibok_2009100812001',
 'URN:NBN:no-nb_digibok_2013110424054',
 'URN:NBN:no-nb_digibok_2009061912006']

In [141]:
dtm = dh.Counts(corpus=urns)

In [150]:
dtm.counts.to_csv('norn_consensus_dtm.csv')

In [167]:
find_urns(dhlabids=list(dtm.counts.columns))

['URN:NBN:no-nb_digibok_2006081700063',
 'URN:NBN:no-nb_digibok_2006082100022',
 'URN:NBN:no-nb_digibok_2006082800104',
 'URN:NBN:no-nb_digibok_2006111400026',
 'URN:NBN:no-nb_digibok_2006112200010',
 'URN:NBN:no-nb_digibok_2007061502002',
 'URN:NBN:no-nb_digibok_2007072402001',
 'URN:NBN:no-nb_digibok_2007082703001',
 'URN:NBN:no-nb_digibok_2007082903001',
 'URN:NBN:no-nb_digibok_2007091202001',
 'URN:NBN:no-nb_digibok_2008011112004',
 'URN:NBN:no-nb_digibok_2008061012003',
 'URN:NBN:no-nb_digibok_2008061012004',
 'URN:NBN:no-nb_digibok_2008061112002',
 'URN:NBN:no-nb_digibok_2008071512001',
 'URN:NBN:no-nb_digibok_2008102112004',
 'URN:NBN:no-nb_digibok_2008111003003',
 'URN:NBN:no-nb_digibok_2008111103001',
 'URN:NBN:no-nb_digibok_2008111103013',
 'URN:NBN:no-nb_digibok_2008111712001',
 'URN:NBN:no-nb_digibok_2008111903007',
 'URN:NBN:no-nb_digibok_2008111903017',
 'URN:NBN:no-nb_digibok_2009061903027',
 'URN:NBN:no-nb_digibok_2009061912006',
 'URN:NBN:no-nb_digibok_2009062212003',
